In [1]:
import xml.etree.ElementTree as elemTree
import os
import cv2
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data.sampler import RandomSampler
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import time
import matplotlib.pyplot as plt
from PIL import ImageDraw

xml_path = 'VOCdevkit/VOC2007/Annotations/' # xml 파일
img_path = 'VOCdevkit/VOC2007/JPEGImages/' # 이미지 파일

img_list = os.listdir(img_path) # 각 이미지 경로 리스트
xml_list = os.listdir(xml_path) # 각 xml파일 경로 리스트

img_list = [f'{img_path}{name}' for name in img_list] # 이미지 경로에 있는 파일명들 리스트
xml_list = [f'{xml_path}{name}' for name in xml_list] # xml 경로에 있는 xml파일들 리스트

file = (img_list,xml_list)        


In [3]:
def get_transforms(image_size):

    transforms_train = transforms.Compose([
                                       transforms.Resize(image_size),
                                       transforms.ToTensor()
                                       ])
    
    transforms_val = transforms.Compose([
                                      transforms.Resize(image_size),
                                      transforms.ToTensor()
                                      ])
    
    return transforms_train, transforms_val

In [4]:
class Classification_Dataset(Dataset):
    def __init__(self, file, mode, transform=None):
        self.img_file = file[0]
        self.xml_file = file[1]
        self.transform = transform
    
    def __len__(self):
        return len(self.img_file)//4  # 데이터 개수

    def __getitem__(self, index):   
        img_row = self.img_file[index] # 주어진 index에 대한 데이터 뽑아오기
        image = Image.open(img_row).convert('RGB') # 파일경로로 부터 이미지를 읽고 rgb로 변환하기
        
        xml_row = self.xml_file[index]
        
        tree = elemTree.parse(xml_row) # XML 로드
        box_lists = [] # labeling한 (x,y,w,h,c) 정보
        label = torch.zeros(7, 7, 30) # label
        
        img_w = int(tree.find('size').findtext('width')) # 이미지 width
        img_h = int(tree.find('size').findtext('height')) # 이미지 height
        
        tag = {'car' : 0, 'chair' : 1, 'person' : 2, 'bicycle' : 3, 'horse' : 4, 'train' : 5, 'aeroplane' : 6, 'diningtable' : 7,
            'dog' : 8, 'tvmonitor' : 9, 'bird' : 10, 'pottedplant' : 11, 'motorbike' : 12, 'boat' : 13, 'sofa' : 14, 'bus' : 15,
            'bottle' : 16, 'sheep' : 17, 'cat' : 18, 'cow' : 19}
        
        for user in tree.findall('object'):
            
            box_list = []
            # 좌측 상단 x,y
            xmin = int(user.find('bndbox').findtext('xmin'))
            ymin = int(user.find('bndbox').findtext('ymin'))
            # 우측 하단 x,y
            xmax = int(user.find('bndbox').findtext('xmax'))
            ymax = int(user.find('bndbox').findtext('ymax'))
            
            # (class_label, x, y, w, h)
            box_list.append(tag[user.findtext('name')]) # 물체 이름에 맞게 라벨링 값으로 넣기
            box_list.append(int((xmax - xmin)/2 + xmin) / img_w * 448) # box의 중심좌표 x
            box_list.append(int((ymax - ymin)/2 + ymin) / img_h * 448) # box의 중심좌표 y
            box_list.append((xmax - xmin) / img_w * 448) # box의 w
            box_list.append((ymax - ymin) / img_h * 448) # box의 h
            
            box_lists.append(box_list) # 실제로 쓸 박스 정보(448*488에 맞게 저장된 박스 정보)
       
        # box_lists를 사용하여 7*7*30 tensor를 만들어보자.
        # 448*448이미지로 resize된거에 맞춰서 정규화
        #box_lists (class_label, x, y, w, h)
        
        for i in range(len(box_lists)):
            grid_col = int(box_lists[i][1]//64) # 몇번째 그리드셀인지 (열)
            grid_row = int(box_lists[i][2]//64) # 몇번째 그리드셀인지 (행)
            label[grid_row, grid_col, box_lists[i][0]] = 1 # 0~19중 몇번째 클래스인지
   
            
            if(label[grid_row, grid_col, 24] == 0):
                label[grid_row, grid_col, 24] = 1 # c_1       
                label[grid_row, grid_col, 20] =  (box_lists[i][1] - (box_lists[i][1] // 64)*64) /64
                label[grid_row, grid_col, 21] =  (box_lists[i][2] - (box_lists[i][2] // 64)*64) /64
                label[grid_row, grid_col, 22] = box_lists[i][3] / 448 # w_1
                label[grid_row, grid_col, 23] = box_lists[i][4] / 448 # h_1

        if self.transform:
            image = self.transform(image) # 이미지에 transform 적용하기

        return image, label  # 이미지와 target return하기

In [5]:
# 모델 트랜스폼 가져오기
transforms_train, transforms_val = get_transforms((448,448))

In [6]:
# dataset class 객체 만들기
dataset_train = Classification_Dataset(file, 'train', transform=transforms_train)
#dataset_val = Classification_Dataset(df_val, 'valid', transform=transforms_val)

In [7]:
print(f'>> 데이터 셋 전체 크기, 개수, sample개수 : {dataset_train.__len__()}')
print(f'>> i번째 데이터 불러오기 : dataset_train.__getitem__(i)')
print(f'>> 0번째 데이터 불러와지는 형태 : tuple (이미지 픽셀값, target)\n : {dataset_train.__getitem__(0)}')

>> 데이터 셋 전체 크기, 개수, sample개수 : 1252
>> i번째 데이터 불러오기 : dataset_train.__getitem__(i)
>> 0번째 데이터 불러와지는 형태 : tuple (이미지 픽셀값, target)
 : (tensor([[[0.0431, 0.0471, 0.0392,  ..., 0.6431, 0.6392, 0.6353],
         [0.0235, 0.0392, 0.0392,  ..., 0.6353, 0.6314, 0.6314],
         [0.1059, 0.0667, 0.0471,  ..., 0.6353, 0.6353, 0.6353],
         ...,
         [0.1333, 0.1294, 0.1255,  ..., 0.2353, 0.2471, 0.2431],
         [0.1608, 0.1647, 0.2078,  ..., 0.2510, 0.2667, 0.2706],
         [0.2000, 0.2510, 0.3098,  ..., 0.2431, 0.2627, 0.2706]],

        [[0.0392, 0.0471, 0.0431,  ..., 0.7333, 0.7294, 0.7255],
         [0.0235, 0.0392, 0.0392,  ..., 0.7373, 0.7373, 0.7333],
         [0.1137, 0.0706, 0.0471,  ..., 0.7412, 0.7412, 0.7412],
         ...,
         [0.0431, 0.0471, 0.0471,  ..., 0.2863, 0.2980, 0.2980],
         [0.0392, 0.0392, 0.0667,  ..., 0.3059, 0.3255, 0.3294],
         [0.0706, 0.0863, 0.1098,  ..., 0.3020, 0.3216, 0.3294]],

        [[0.0392, 0.0471, 0.0431,  ..., 0.7569, 0.7529,

In [8]:
dataset_train.__getitem__(0)[0].shape

torch.Size([3, 448, 448])

In [9]:
dataset_train.__getitem__(0)[1].shape

torch.Size([7, 7, 30])

In [10]:
batch_size = 16
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, sampler=RandomSampler(dataset_train), num_workers=0)
# valid_loader = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, num_workers=0)

In [11]:
for images, targets in train_loader:
    print(f'[batch, channel, ...] : {images.shape}')
    print(targets.shape)
    break

[batch, channel, ...] : torch.Size([16, 3, 448, 448])
torch.Size([16, 7, 7, 30])


In [12]:
# Model
class Net(nn.Module):
    def __init__(self):
        # 3 X 448 X 448
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3, 64, 7, padding=3,stride=2), 
                                    # 64 X 224 X 224
                                    nn.BatchNorm2d(64),
                                    nn.LeakyReLU(0.1), 
                                    nn.MaxPool2d(2,2)
                                    # 64 X 112 X 112
                                   )
        self.layer2 = nn.Sequential(nn.Conv2d(64, 192, 3, padding=1), 
                                    # 192 X 112 X 112
                                    nn.BatchNorm2d(192),
                                    nn.LeakyReLU(0.1), 
                                    nn.MaxPool2d(2,2)
                                    # 192 X 56 X 56
                                   )
        self.layer3 = nn.Sequential(nn.Conv2d(192, 128, 1), 
                                    # 128 X 112 X 112
                                    nn.BatchNorm2d(128),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(128, 256, 3, padding=1),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(256, 256, 1),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(256, 512, 3, padding=1),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1),
                                    nn.MaxPool2d(2,2)
                               )
        self.layer4 = nn.Sequential(nn.Conv2d(512, 256, 1), 
                                    # 128 X 112 X 112
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(256, 512, 3, padding=1),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(512, 256, 1),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(256, 512, 3, padding=1),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(512, 256, 1), 
                                    nn.BatchNorm2d(256),
                                    # 128 X 112 X 112
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(256, 512, 3, padding=1),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(512, 256, 1),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(256, 512, 3, padding=1),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(512, 512, 1),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(512, 1024, 3, padding=1),
                                    nn.BatchNorm2d(1024),
                                    nn.LeakyReLU(0.1),
                                    nn.MaxPool2d(2,2)
                               )
        self.layer5 = nn.Sequential(nn.Conv2d(1024, 512, 1),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(512, 1024, 3, padding=1),
                                    nn.BatchNorm2d(1024),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(1024, 512, 1),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(512, 1024, 3, padding=1),
                                    nn.BatchNorm2d(1024),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(1024, 1024, 3, padding=1),
                                    nn.BatchNorm2d(1024),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(1024, 1024, 3, padding=1, stride=2),
                                    nn.BatchNorm2d(1024),
                                    nn.LeakyReLU(0.1)
                               )
        self.layer6 = nn.Sequential(nn.Conv2d(1024, 1024, 3, padding=1),
                                    nn.BatchNorm2d(1024),
                                    nn.LeakyReLU(0.1),
                                    nn.Conv2d(1024, 1024, 3, padding=1),
                                    nn.BatchNorm2d(1024),
                                    nn.LeakyReLU(0.1),
                                )

        self.layer7 = nn.Sequential(nn.Linear(1024*7*7, 4096),
                                    nn.Dropout(0.5),
                                    nn.LeakyReLU(0.1),
                                    nn.Linear(4096, 7*7*30),                            
                                    #nn.Sigmoid()
                                    )
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')

    def forward(self, x):
        x = self.layer1(x) 
        x = self.layer2(x)   
        x = self.layer3(x) 
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = torch.flatten(x,1) 
        x = self.layer7(x)
        x = x.reshape(-1,7,7,30)
        
        return x

In [13]:
def IOU(box1, box2):
    # box1 = (..., (x, y, w, h))
    # box2 = (..., (x, y, w, h))
    box1_min_coord = (box1[..., 0]-(box1[..., 2]/2), box1[..., 1]-(box1[..., 3]/2)) # 좌측 하단의 좌표 (x,y가 최소값)
    box1_max_coord = (box1[..., 0]+(box1[..., 2]/2), box1[..., 1]+(box1[..., 3]/2)) # 우측 상단의 좌표 (x,y가 최대값)
    box2_min_coord = (box2[..., 0]-(box1[..., 2]/2), box2[..., 1]-(box1[..., 3]/2)) # 좌측 하단의 좌표 (x,y가 최소값)
    box2_max_coord = (box2[..., 0]+(box1[..., 2]/2), box2[..., 1]+(box1[..., 3]/2)) # 우측 상단의 좌표 (x,y가 최대값)
    # tuple( (batch, 7, 7) , (batch, 7, 7))

    box1_area = box1[..., 2] * box1[..., 3] # box1의 넓이
    box2_area = box2[..., 2] * box2[..., 3] # box2의 넓이

    inter_w = cv2.min(box1_max_coord[0].detach().cpu().numpy(), box2_max_coord[0].detach().cpu().numpy()) - cv2.max(box1_min_coord[0].detach().cpu().numpy(), box2_min_coord[0].detach().cpu().numpy())
    inter_h = cv2.min(box1_max_coord[1].detach().cpu().numpy(), box2_max_coord[1].detach().cpu().numpy()) - cv2.max(box1_min_coord[1].detach().cpu().numpy(), box2_min_coord[1].detach().cpu().numpy())
    
    inter_area = torch.tensor(inter_w * inter_h).to(device)
    
    iou = (inter_area / (box1_area + box2_area - inter_area))
    
    return iou

In [14]:
# sig, cross entropy loss
class Loss(nn.Module):
    def __init__(self, S=7, B=2, C=20):
        super(Loss, self).__init__()
        self.crosloss = nn.MSELoss()
        self.S = S
        self.B = B
        self.C = C
        self.lambda_noobj = 0.5
        self.lambda_coord = 5
    
    def forward(self, pred, target):
        
        iou_b1 = IOU(pred[..., 20:24], target[..., 20:24]) # (x_1, y_1, w_1, h_1)
        iou_b2 = IOU(pred[..., 25:29], target[..., 20:24]) # (x_2, y_2, w_2, h_2)
        
        bestbox = (iou_b1 > iou_b2).reshape(-1,7,7,1) # iou값이 b1이 크면 1, b2가 크면 0
        
        exist_box = target[..., 24].unsqueeze(3) # 4x7x7x1 (ground truth box의 중심 좌표가 해당 그리드 셀에 존재하면 1 아니면 0)

        box_pred = exist_box * (
        (
                bestbox * pred[..., 20:24]
            + (~bestbox) * pred[..., 21:25]
        )
    )
        box_target = target[..., 20:24] * exist_box
        
        # x,y에 대한 loss
        box_x_pred = box_pred[..., [0]]
        box_x_target = box_target[..., [0]]
        box_y_pred = box_pred[..., [1]]
        box_y_target = box_target[..., [1]]
       
        box_x_loss = self.crosloss(box_x_pred, box_x_target)
        box_y_loss = self.crosloss(box_y_pred, box_y_target)
        
        
        # w,h에 대한 loss
        box_w_pred = torch.sign(box_pred[..., 2]) * torch.sqrt(
            torch.abs(box_pred[..., 2] + 1e-6)
            )
        box_w_target = torch.sqrt(box_target[..., [2]])
        box_h_pred = torch.sign(box_pred[..., 3]) * torch.sqrt(
            torch.abs(box_pred[..., 3] + 1e-6)
            )
        box_h_target = torch.sqrt(box_target[..., [3]])

        box_w_loss = self.crosloss(box_w_pred.reshape(-1,7,7,1), box_w_target)
        box_h_loss = self.crosloss(box_h_pred.reshape(-1,7,7,1), box_h_target)
        
        # confidence loss
        # 존재할때 
        pred_box = bestbox * pred[..., 24:25] + (~bestbox) * pred[..., 29:30]
        obj_conf_loss = self.crosloss((exist_box*pred_box), (exist_box*target[...,24].reshape(-1,7,7,1)))
        
        #존재하지 않을 때
        noobj_conf_loss = self.crosloss((1-exist_box)*pred[..., 24:25],(1-exist_box)*target[..., 24].reshape(-1,7,7,1))
        noobj_conf_loss += self.crosloss((1-exist_box)*pred[..., 29:30],(1-exist_box)*target[..., 24].reshape(-1,7,7,1))
        class_loss = 0.
        
        # classification loss
        for i in range(20):
            class_loss += self.crosloss((exist_box * pred[...,[i]]), (exist_box*target[...,[i]]))
        
        if torch.isnan(box_x_loss).detach().cpu().numpy() == True :
            box_x_loss = torch.zeros_like(box_x_loss)
        if torch.isnan(box_y_loss).detach().cpu().numpy() == True :
            box_y_loss = torch.zeros_like(box_y_loss)
        if torch.isnan(box_w_loss).detach().cpu().numpy() == True :
            box_w_loss = torch.zeros_like(box_w_loss)
        if torch.isnan(box_h_loss).detach().cpu().numpy() == True :
            box_h_loss = torch.zeros_like(box_h_loss)
        
#         # nan 방지
#         if torch.isnan(box_wh_loss).detach().cpu().numpy() == True :
#             box_wh_loss = torch.zeros_like(box_wh_loss)
            
        loss = (self.lambda_coord * (box_x_loss + box_y_loss + box_w_loss + box_h_loss)
                + obj_conf_loss + self.lambda_noobj * noobj_conf_loss 
                + class_loss) 
        return loss
#         print(f'x,y = {self.lambda_coord * box_xy_loss} // w,h = {self.lambda_coord * box_wh_loss} // obj_conf = {obj_conf_loss} // \
#               noobj_conf = {self.lambda_noobj * noobj_conf_loss} // classifi = {class_loss} // total loss =  {loss}')        
#         return loss

In [15]:
learning_rate = 0.001
num_epoch = 135
weight_decay = 5.0e-4
momentum=0.9

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = Net().to(device)

loss_func = Loss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=75, gamma=0.1)

cuda:0


In [18]:
for i in range(num_epoch):
    total_loss = 0.
    count = 0
    print(f'epoch: {i+1}/{num_epoch}')
    for image, target in (train_loader):            
        optimizer.zero_grad()
        image = image.to(device)     
        target = target.to(device)
        output = model(image)
        
        loss = loss_func(output, target) 
        total_loss += loss.item()
        count += 1

        print(loss.item())
        loss.backward()
        optimizer.step()

        torch.save(model.state_dict(), f'best_model_BCE_{i}.pth')
    content = time.ctime() + ' ' + f'Epoch {i}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {total_loss/count:.5f}'
    scheduler.step()
    print(content)
        
        
        

epoch: 1/135
0.0864500179886818
0.0831146240234375


RuntimeError: [enforce fail at ..\caffe2\serialize\inline_container.cc:300] . unexpected pos 212155968 vs 212155864

0~19: classification  
20,21,22,23,24 : x,y,w,h,c

In [35]:
# model_ = torch.load('best_model.pth')
# model_.eval()
threshold = 0.1
iou_threshol = 0.5
model = Net()
model.load_state_dict(torch.load('best_model_BCE_134.pth'))
model.eval()
for (image, target) in (train_loader):
    pre = model(image)
    print(pre.shape)
    
    for i in range(pre.shape[0]): # batch
        for j in range(pre.shape[1]): # grid col
            for k in range(pre.shape[2]): # grid row
                if pre[i][j][k][24] > threshold :
                    b1_x = int(64*j + 64*pre[i][j][k][20])
                    b1_y = int(64*k + 64*pre[i][j][k][21])
                    b1_w = int(448 * pre[i][j][k][22])
                    b1_h = int(448 * pre[i][j][k][23])
                    b1_x_min = int(b1_x - (b1_w/2))
                    b1_y_min = int(b1_y - (b1_h/2))
                    b1_x_max = int(b1_x + (b1_w/2))
                    b1_y_max = int(b1_y + (b1_h/2))
                    # 바운딩박스 그리기
                    images = cv2.rectangle((255*torch.permute(image[i],(1,2,0)).detach().cpu().numpy()).astype(np.uint8).copy(),\
                                           (b1_x_min, b1_y_min),(b1_x_max,b1_y_max),color=(255,0,0), thickness=1) 
                    cv2.imshow('show results',images)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                if pre[i][j][k][29] > threshold :
                    b2_x = int(64*j + 64*pre[i][j][k][25])
                    b2_y = int(64*k + 64*pre[i][j][k][26])
                    b2_w = int(448 * pre[i][j][k][27])
                    b2_h = int(448 * pre[i][j][k][28])

                    b2_x_min = int(b2_x - (b2_w/2))
                    b2_y_min = int(b2_y - (b2_h/2))
                    b2_x_max = int(b2_x + (b2_w/2))
                    b2_y_max = int(b2_y + (b2_h/2))
                    # 바운딩박스 그리기
                    images = cv2.rectangle((255*torch.permute(image[i],(1,2,0)).detach().cpu().numpy()).astype(np.uint8).copy(),\
                                           (b2_x_min, b2_y_min),(b2_x_max,b2_y_max),color=(255,0,0), thickness=1) 
                    cv2.imshow('show results',images)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                    
                    
                    
                #print(torch.permute(image[i],(1,2,0))*255)
                #image = cv2.putText(image, bb_list[i][0], (bb_list[i][1], bb_list[i][2] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), thickness=3)
                


torch.Size([16, 7, 7, 30])


KeyboardInterrupt: 

In [33]:
threshold = 0.1
iou_threshol = 0.5
model = Net()
model.load_state_dict(torch.load('best_model_BCE_134.pth'))
model.eval()
bb_list = torch.tensor([])
for (image, target) in (train_loader):
    pre = model(image)
    print(pre.shape)
    
    for i in range(pre.shape[0]): # batch
        for j in range(pre.shape[1]): # grid col
            for k in range(pre.shape[2]): # grid row
                
                pre[i][j][k][:20] # class 0~19
                pre[i][j][k][24] # box1_c
                pre[i][j][k][29] # box2_c
                
                
                bb_temp = torch.hstack(((pre[i][j][k][:20] * pre[i][j][k][24]).reshape(-1,1), (pre[i][j][k][:20] * pre[i][j][k][29]).reshape(-1,1))) 
                torch.hstack((bb_list, bb_temp))
                print(bb_list)
                
#                 if pre[i][j][k][24] > threshold :
#                     b1_x = int(64*j + 64*pre[i][j][k][20])
#                     b1_y = int(64*k + 64*pre[i][j][k][21])
#                     b1_w = int(448 * pre[i][j][k][22])
#                     b1_h = int(448 * pre[i][j][k][23])
#                     b1_x_min = int(b1_x - (b1_w/2))
#                     b1_y_min = int(b1_y - (b1_h/2))
#                     b1_x_max = int(b1_x + (b1_w/2))
#                     b1_y_max = int(b1_y + (b1_h/2))
#                     # 바운딩박스 그리기
#                     images = cv2.rectangle((255*torch.permute(image[i],(1,2,0)).detach().cpu().numpy()).astype(np.uint8).copy(),\
#                                            (b1_x_min, b1_y_min),(b1_x_max,b1_y_max),color=(255,0,0), thickness=1) 
#                     cv2.imshow('show results',images)
#                     cv2.waitKey(0)
#                     cv2.destroyAllWindows()
#                 if pre[i][j][k][29] > threshold :
#                     b2_x = int(64*j + 64*pre[i][j][k][25])
#                     b2_y = int(64*k + 64*pre[i][j][k][26])
#                     b2_w = int(448 * pre[i][j][k][27])
#                     b2_h = int(448 * pre[i][j][k][28])

#                     b2_x_min = int(b2_x - (b2_w/2))
#                     b2_y_min = int(b2_y - (b2_h/2))
#                     b2_x_max = int(b2_x + (b2_w/2))
#                     b2_y_max = int(b2_y + (b2_h/2))
#                     # 바운딩박스 그리기
#                     images = cv2.rectangle((255*torch.permute(image[i],(1,2,0)).detach().cpu().numpy()).astype(np.uint8).copy(),\
#                                            (b2_x_min, b2_y_min),(b2_x_max,b2_y_max),color=(255,0,0), thickness=1) 
#                     cv2.imshow('show results',images)
#                     cv2.waitKey(0)
#                     cv2.destroyAllWindows()
                    
                    
                    
#                 #print(torch.permute(image[i],(1,2,0))*255)
#                 #image = cv2.putText(image, bb_list[i][0], (bb_list[i][1], bb_list[i][2] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), thickness=3)
                




torch.Size([16, 7, 7, 30])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tenso

torch.Size([16, 7, 7, 30])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tenso

torch.Size([16, 7, 7, 30])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tensor([])
tenso

torch.Size([16, 7, 7, 30])


KeyboardInterrupt: 